In [63]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import os, os.path
from keras.models import load_model
import traceback


In [88]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize webcam
capture = cv2.VideoCapture(0)
if not capture.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Assuming `hd` is your custom hand detection model, initialized elsewhere
# hd = SomeHandDetectionClass()  # Ensure hd is properly initialized

# Offset for cropping
offset = 20  # Adjust as needed

while True:
    try:
        # Capture frame
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)  # Flip the frame horizontally

        # Detect hands using mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks using mediapipe
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Get bounding box from the landmarks (if needed)
                x_min = min([landmark.x for landmark in landmarks.landmark])
                x_max = max([landmark.x for landmark in landmarks.landmark])
                y_min = min([landmark.y for landmark in landmarks.landmark])
                y_max = max([landmark.y for landmark in landmarks.landmark])

                # Convert to pixel coordinates
                height, width, _ = frame.shape
                x_min, x_max = int(x_min * width), int(x_max * width)
                y_min, y_max = int(y_min * height), int(y_max * height)

                # Crop the hand region from the frame
                hand_region = frame[y_min - offset:y_max + offset, x_min - offset:x_max + offset]

                # Process hand region (gray, thresholding, etc.)
                gray = cv2.cvtColor(hand_region, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(gray, (5, 5), 2)
                th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
                ret, test_image = cv2.threshold(th3, 27, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Prepare the final images with padding
                img_final = np.ones((400, 400), np.uint8) * 255
                h, w = test_image.shape
                img_final[((400 - h) // 2):((400 - h) // 2) + h, ((400 - w) // 2):((400 - w) // 2) + w] = test_image

                # Process skeleton using hd (assuming hd.findHands() works similarly to mediapipe)
                handz = hd.findHands(hand_region, draw=False, flipType=True)
                
                if handz:
                    # Check if handz is a list and contains a dictionary with the expected keys
                    hand = handz[0]
                    
                    # Check if the keys exist
                    if 'lmList' in hand and 'bbox' in hand:
                        pts = hand['lmList']
                        x, y, w, h = hand['bbox']

                        # Adjust the offset if necessary
                        os = ((400 - w) // 2) - 15
                        os1 = ((400 - h) // 2) - 15

                        # Drawing the skeleton using hd landmarks
                        white = cv2.imread("white.jpg")  # White background for skeleton
                        for t in range(0, 4, 1):
                            cv2.line(white, (pts[t][0] + os, pts[t][1] + os1), (pts[t + 1][0] + os, pts[t + 1][1] + os1), (0, 255, 0), 3)
                        for t in range(5, 8, 1):
                            cv2.line(white, (pts[t][0] + os, pts[t][1] + os1), (pts[t + 1][0] + os, pts[t + 1][1] + os1), (0, 255, 0), 3)
                        # Add more lines for the hand skeleton as necessary

                        # Show skeleton image
                        cv2.imshow("Hand Skeleton", white)

                # Display the processed images
                cv2.imshow("Processed Hand", img_final)
                cv2.imshow("Original Frame", frame)

        # Wait for key press
        interrupt = cv2.waitKey(1)
        if interrupt & 0xFF == 27:  # Escape key to exit
            break

    except Exception as e:
        print(f"Error: {e}")
        break

# Release webcam and close all windows
capture.release()
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import mediapipe as mp
from keras.models import load_model

# Load the pre-trained model
model = load_model(r'C:\Users\Darsh\Desktop\Project\mp\Code\asl_classifier_new.h5')

# Initialize webcam
capture = cv2.VideoCapture(0)
if not capture.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Offset for cropping
offset = 20  # Adjust as needed

def preprocess_image(img):
    """Resize and prepare the image for prediction."""
    img_resized = cv2.resize(img, (128, 128))  # Resize to 128x128 as expected by the model
    img_resized = img_resized / 255.0  # Normalize to [0, 1]
    img_resized = np.expand_dims(img_resized, axis=-1)  # Add channel dimension for grayscale
    img_resized = np.expand_dims(img_resized, axis=0)   # Add batch dimension
    return img_resized



while True:
    try:
        # Capture frame
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)  # Flip the frame horizontally

        # Detect hands using MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks using MediaPipe
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Get bounding box from the landmarks (if needed)
                x_min = min([landmark.x for landmark in landmarks.landmark])
                x_max = max([landmark.x for landmark in landmarks.landmark])
                y_min = min([landmark.y for landmark in landmarks.landmark])
                y_max = max([landmark.y for landmark in landmarks.landmark])

                # Convert to pixel coordinates
                height, width, _ = frame.shape
                x_min, x_max = int(x_min * width), int(x_max * width)
                y_min, y_max = int(y_min * height), int(y_max * height)

                # Crop the hand region from the frame
                hand_region = frame[y_min - offset:y_max + offset, x_min - offset:x_max + offset]

                # Process hand region (gray, thresholding, etc.)
                gray = cv2.cvtColor(hand_region, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(gray, (5, 5), 2)
                th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
                ret, test_image = cv2.threshold(th3, 27, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Prepare the final image with padding
                img_final = np.ones((400, 400), np.uint8) * 255
                h, w = test_image.shape
                img_final[((400 - h) // 2):((400 - h) // 2) + h, ((400 - w) // 2):((400 - w) // 2) + w] = test_image

                # Preprocess the image for the model
                input_image = preprocess_image(img_final)
                
                # Use the model to predict the gesture
                predictions = model.predict(input_image)
                predicted_class = np.argmax(predictions)  # Get the class with highest probability
                gesture_label = chr(predicted_class + 65)  # Assuming 0=A, 1=B, etc.

                # Display the predicted gesture on the frame
                cv2.putText(frame, f"Prediction: {gesture_label}", (10, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display the processed images
                cv2.imshow("Processed Hand", img_final)
                cv2.imshow("Original Frame", frame)

        # Wait for key press
        interrupt = cv2.waitKey(1)
        if interrupt & 0xFF == 27:  # Escape key to exit
            break

    except Exception as e:
        print(f"Error: {e}")
        break

# Release webcam and close all windows
capture.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Predictions: [[0.02901051 0.05201129 0.06244478 0.02089472 0.49110454 0.04564295
  0.25776255 0.04112861]]


In [6]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Initialize webcam
capture = cv2.VideoCapture(0)
if not capture.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Load the custom model (update with the correct path to your model)
hd = tf.keras.models.load_model(r"C:\Users\Darsh\Desktop\Project\mp\Code\asl_classifier_new.h5")

# Offset for cropping
offset = 20  # Adjust as needed

while True:
    try:
        # Capture frame
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)  # Flip the frame horizontally

        # Detect hands using mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Get bounding box coordinates
                x_min = min([landmark.x for landmark in landmarks.landmark])
                x_max = max([landmark.x for landmark in landmarks.landmark])
                y_min = min([landmark.y for landmark in landmarks.landmark])
                y_max = max([landmark.y for landmark in landmarks.landmark])

                # Convert to pixel coordinates
                height, width, _ = frame.shape
                x_min, x_max = int(x_min * width), int(x_max * width)
                y_min, y_max = int(y_min * height), int(y_max * height)

                # Crop the hand region from the frame
                hand_region = frame[y_min - offset:y_max + offset, x_min - offset:x_max + offset]

                # Process hand region (gray, thresholding, etc.)
                gray = cv2.cvtColor(hand_region, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(gray, (5, 5), 2)
                th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
                ret, test_image = cv2.threshold(th3, 27, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Prepare the final images with padding
                img_final = np.ones((400, 400), np.uint8) * 255
                h, w = test_image.shape
                img_final[((400 - h) // 2):((400 - h) // 2) + h, ((400 - w) // 2):((400 - w) // 2) + w] = test_image

                # Preprocess for the model: resize to expected input shape and add batch dimension
                input_image = cv2.resize(img_final, (128, 128))  # Assuming the model input size is 224x224

                # If your model expects RGB input (3 channels), convert grayscale to RGB
                input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2RGB)

                input_image = input_image / 255.0  # Normalize if the model expects [0, 1] range
                input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

                # Make the prediction
                prediction = hd.predict(input_image)

                # Process and display the prediction
                predicted_class = np.argmax(prediction)  # Get the class with the highest probability
                confidence = np.max(prediction)  # Get confidence of prediction

                # Display the prediction on the frame
                cv2.putText(frame, f"Class: {predicted_class} (Confidence: {confidence:.2f})", 
                            (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Show processed images
                cv2.imshow("Processed Hand", img_final)
                cv2.imshow("Original Frame", frame)

        # Wait for key press
        interrupt = cv2.waitKey(1)
        if interrupt & 0xFF == 27:  # Escape key to exit
            break

    except Exception as e:
        print(f"Error: {e}")
        break

# Release webcam and close all windows
capture.release()
cv2.destroyAllWindows()


Error: Exception encountered when calling Sequential.call().

Input 0 of layer "conv2d_4" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 128, 128, 3)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 128, 128, 3), dtype=float32)
  • training=False
  • mask=None


In [91]:
hd.summary()


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 398, 398, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 199, 199, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 197, 197, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 98, 98, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 96, 96, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 48, 48, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 46, 46, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 23, 23, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8464)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │     1,083,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 96)             │        12,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,119,722 (4.27 MB)

 Trainable params: 1,119,720 (4.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [8]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Initialize webcam
capture = cv2.VideoCapture(0)
if not capture.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Load the custom model (update with the correct path to your model)
hd = tf.keras.models.load_model(r"C:\Users\Darsh\Desktop\Project\mp\Code\asl_classifier_new.h5")

# Offset for cropping
offset = 20  # Adjust as needed

while True:
    try:
        # Capture frame
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)  # Flip the frame horizontally

        # Detect hands using mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Get bounding box coordinates
                x_min = min([landmark.x for landmark in landmarks.landmark])
                x_max = max([landmark.x for landmark in landmarks.landmark])
                y_min = min([landmark.y for landmark in landmarks.landmark])
                y_max = max([landmark.y for landmark in landmarks.landmark])

                # Convert to pixel coordinates
                height, width, _ = frame.shape
                x_min, x_max = int(x_min * width), int(x_max * width)
                y_min, y_max = int(y_min * height), int(y_max * height)

                # Crop the hand region from the frame
                hand_region = frame[y_min - offset:y_max + offset, x_min - offset:x_max + offset]

                # Process hand region (gray, thresholding, etc.)
                gray = cv2.cvtColor(hand_region, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(gray, (5, 5), 2)
                th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
                ret, test_image = cv2.threshold(th3, 27, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Prepare the final image with padding
                img_final = np.ones((400, 400), np.uint8) * 255
                h, w = test_image.shape
                img_final[((400 - h) // 2):((400 - h) // 2) + h, ((400 - w) // 2):((400 - w) // 2) + w] = test_image

                # Preprocess for the model: resize to expected input size (224x224), ensure 3 channels (RGB)
                input_image = cv2.resize(img_final, (128, 128))  # Resize to 224x224
                input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2RGB)  # Ensure 3 channels
                input_image = input_image / 255.0  # Normalize the image to [0, 1]
                input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension

                # Make the prediction
                prediction = hd.predict(input_image)
                
                # Process and display the prediction
                predicted_class = np.argmax(prediction)  # Get the class with the highest probability
                confidence = np.max(prediction)  # Get confidence of prediction

                # Display the prediction on the frame
                cv2.putText(frame, f"Class: {predicted_class} (Confidence: {confidence:.2f})", 
                            (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Show processed images
                cv2.imshow("Processed Hand", img_final)
                cv2.imshow("Original Frame", frame)

        # Wait for key press
        interrupt = cv2.waitKey(1)
        if interrupt & 0xFF == 27:  # Escape key to exit
            break

    except Exception as e:
        print(f"Error: {e}")
        break

# Release webcam and close all windows
capture.release()
cv2.destroyAllWindows()


Error: Exception encountered when calling Sequential.call().

Input 0 of layer "conv2d_4" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 128, 128, 3)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 128, 128, 3), dtype=float32)
  • training=False
  • mask=None


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

# Initialize webcam
capture = cv2.VideoCapture(0)
if not capture.isOpened():
    print("Error: Unable to access the webcam.")
    exit()

# Initialize MediaPipe hand detector
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Load the custom model (update with the correct path to your model)
hd = tf.keras.models.load_model(r"C:\Users\Darsh\Desktop\Project\mp\Code\asl_classifier_new.h5")

# Offset for cropping
offset = 20  # Adjust as needed

while True:
    try:
        # Capture frame
        ret, frame = capture.read()
        if not ret:
            print("Failed to capture frame.")
            break

        frame = cv2.flip(frame, 1)  # Flip the frame horizontally

        # Detect hands using mediapipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks
                mp_drawing.draw_landmarks(frame, landmarks, mp_hands.HAND_CONNECTIONS)

                # Get bounding box coordinates
                x_min = min([landmark.x for landmark in landmarks.landmark])
                x_max = max([landmark.x for landmark in landmarks.landmark])
                y_min = min([landmark.y for landmark in landmarks.landmark])
                y_max = max([landmark.y for landmark in landmarks.landmark])

                # Convert to pixel coordinates
                height, width, _ = frame.shape
                x_min, x_max = int(x_min * width), int(x_max * width)
                y_min, y_max = int(y_min * height), int(y_max * height)

                # Crop the hand region from the frame
                hand_region = frame[y_min - offset:y_max + offset, x_min - offset:x_max + offset]

                # Process hand region (gray, thresholding, etc.)
                gray = cv2.cvtColor(hand_region, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(gray, (5, 5), 2)
                th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
                ret, test_image = cv2.threshold(th3, 27, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

                # Prepare the final images with padding
                img_final = np.ones((400, 400), np.uint8) * 255
                h, w = test_image.shape
                img_final[((400 - h) // 2):((400 - h) // 2) + h, ((400 - w) // 2):((400 - w) // 2) + w] = test_image

                # Preprocess for the model: resize to (400, 400) and convert to RGB
                input_image = cv2.resize(img_final, (400, 400))  # Resize to (400, 400)
                input_image = cv2.cvtColor(input_image, cv2.COLOR_GRAY2RGB)  # Convert to RGB
                input_image = input_image / 255.0  # Normalize the image to [0, 1]

                # Add batch dimension
                input_image = np.expand_dims(input_image, axis=0)  # Shape becomes (1, 400, 400, 3)

                # Make the prediction
                prediction = hd.predict(input_image)

                # Process and display the prediction
                predicted_class = np.argmax(prediction)  # Get the class with the highest probability
                confidence = np.max(prediction)  # Get confidence of prediction

                # Display the prediction on the frame
                cv2.putText(frame, f"Class: {predicted_class} (Confidence: {confidence:.2f})", 
                            (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

                # Show processed images
                cv2.imshow("Processed Hand", img_final)
                cv2.imshow("Original Frame", frame)

        # Wait for key press
        interrupt = cv2.waitKey(1)
        if interrupt & 0xFF == 27:  # Escape key to exit
            break

    except Exception as e:
        print(f"Error: {e}")
        break

# Release webcam and close all windows
capture.release()
cv2.destroyAllWindows()


Error: Exception encountered when calling Sequential.call().

Input 0 of layer "conv2d_4" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (1, 400, 400, 3)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 400, 400, 3), dtype=float32)
  • training=False
  • mask=None


Error: Image not loaded. Please check the file path.
